In [ ]:
using Revise; 
includet("TrajModules.jl")
using .atom_class
using .SystemSetup
using .utils
using .BeamClass
using .CoolTrap
using .IntegrateODE

In [ ]:
using ProfileView
using BenchmarkTools
using PyPlot

## Creating the system and light

In [1]:
function createSystem()
    BField(x, y, z) = 800E-4 * [0, 0, 1]
    atomType = cesiumAtom
    OurSystem = System(atomType, BField, -4e-6:0.4e-7:4e-6)
    return OurSystem
end
OurSystem = createSystem();

LoadError: UndefVarError: `cesiumAtom` not defined

In [ ]:
function createTweezer(OurSystem)
    clear_Tweezer(OurSystem)
    stateI_T = (6, 1, 1 / 2)
    stateF_T = (6, 0, 1 / 2)
    wavelength_tweezer = 1064E-9
    trapR, trapZ = 2 * pi * 0.150e6, 2 * pi * 0.029e6
    
    LocAll::Vector{Float64} = [0, 0, 0]
    println("Setting up Tweezer")
    set_tweezer(OurSystem, BeamProperties(LocAll, [0.0, 0.0, 1.0], pol_v["pi"]), stateI_T, stateF_T, wavelength_tweezer, trapR, trapZ)
end
createTweezer(OurSystem);

In [ ]:
function createMOT(OurSystem)
    clear_MOT(OurSystem)
    stateF = (6, 1, 1.5, 5, 5)
    stateI = (6, 0, 0.5, 4, 4)
    detuning = 3e6
    I_mot = 14
    beamAngle = 45 * pi / 180


    println("Setting up MOT")
    updates0 = false
    LocAll::Vector{Float64} = [0, 0, 0]
    set_MOT(OurSystem, BeamProperties(LocAll, [-1.0, 0.0, 0.0], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot, updates0)
    set_MOT(OurSystem, BeamProperties(LocAll, [1.0, 0.0, 0.0], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
    #R1
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, sin(beamAngle), cos(beamAngle)], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot,updates0)
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, -sin(beamAngle), -cos(beamAngle)], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
    #R2
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, -sin(beamAngle), cos(beamAngle)], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot, updates0)
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, sin(beamAngle), -cos(beamAngle)], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
end

createMOT(OurSystem)

In [ ]:
clear_beams(OurSystem)
join_beams(OurSystem)
println("Number of Total Beams: ", length(OurSystem.BeamConfig))
println("Number of Tweezers Beams: ", length(OurSystem.TweezerConfig))
println("Number of MOT Beams: ", length(OurSystem.MOTConfig))

# Simulation 

In [ ]:
dt = 2e-9
tspan = (0, 4e-3)
sol = simulate(OurSystem, 1, 1e-3, "Fabs_spont_R", tspan, dt)

In [ ]:
timeEval, x, y, z, vx, vy, vz, photons = processData(sol)

In [ ]:
fig = plt.figure()
plot(timeEval*1e3, x[1]*1e6)
fig